# Домашняя работа по регуляризации и оптимизации

Ниже приводится корпус данных с двумя метками: 1 и -1. К данным применяется линейная модель классификации:

$f(x, \theta) = x_1 \theta_1 + x_2 \theta_2 + \theta_3.$

Предлагается подобрать параметры $\theta$ минимизируя следующую функцию ошибки:

$\mathcal{L}(\theta) = 0.1 (\theta_1^2 + \theta_2^2) + \frac{1}{N}\sum\limits_{i=1}^N \max(0, 1 - y_i f(x_i, \theta)).$

Для оптимизации предлагается использовать метод градиентного спуска с 1000 шагами размера $0.1$ из начальной точки $(1, 1, 0)$.

Useful liniks: https://en.wikipedia.org/wiki/Hinge_loss, https://en.m.wikipedia.org/wiki/Ridge_regression, https://en.wikipedia.org/wiki/Support_vector_machine, https://habr.com/ru/company/ods/blog/484148/

Для удобства решения загоним bias в матрицу данных, то есть для одного семпла:
$$
x_1 \theta_1 + x_2 \theta_2 + \theta_3 =
\begin{pmatrix}
x_1 & x_2 & 1\\
\end{pmatrix} \cdot
\begin{pmatrix}
\theta_1 \\
\theta_2 \\
\theta_3
\end{pmatrix} = \langle \theta, x_{new} \rangle = g(x_{new}, \theta)
$$

Где $\theta = (\theta_1 \ \theta_2 \ \theta_3)^T, x_{new} = (x_1 \ x_2 \ 1)^T$.

Для градиентного спуска необходимо найти градиент функции ошибок, но сначала преобразуем её к более красивому виду:

$$
\mathcal{L}(\theta) = 0.1 ||\theta||_2^2  -0.1 \cdot \theta_3^2+ \frac{1}{5}\sum\limits_{i=1}^5 H_i(M_i) =
$$

$$
= 0.1 \langle \theta, \theta \rangle  -0.1 \cdot \theta_3^2 + \frac{1}{5}\sum\limits_{i=1}^5 H_i(M_i)
$$

Где $M_i = y_i g(x_{new,i}, \theta) = y_i \langle x_{new,i}, \theta \rangle $ - отступ (margin) (тут в данном случае $i$ означает индекс семпла, а не значение координаты вектора, тем более, что $M_i, y_i$ - это числа), $H(M)$ - hinge loss.

Как известно:

$$
{\frac  {\partial H_i }{\partial \theta_{j}}}={\begin{cases} - y_i \cdot (x_{new, i})_j&{\text{if }}M_i < 1\\
0, &{\text{otherwise}}\end{cases}} = -y_i \cdot (x_{new,i})_j \cdot [M_i < 1]
$$

Поясню это словами: мы берем $i$ семпл (вектор признаков $x_{new,i}$ из нашей матрицы $X$) и для того чтобы найти производную по $j$ координате ($j = \{1, 2, 3\}$) смотрим на значение марджина $M_i$ и в зависимости от него получим, либо 0, либо значение целевой метки для данного семпла умноженную на отрицательное значение $j$ компоненты нашего вектора признаков $x_{new,i}$ .

Обощая на все три координаты, получим:

$$
\nabla_{\theta} H_i = -y_i \cdot [M_i < 1] \cdot x_{new,i}
$$

Тут $i$ - это тоже номер семпла из всей матрицы объектов $X$.

Таким образом для $i$ компоненты градиента:

$$
(\nabla_{\theta} \mathcal{L}(\theta))_i =  0.2 \cdot \theta_i - 0.2 \cdot \theta_i \delta_{i3} + \frac{1}{5}\sum\limits_{j=1}^5\partial_i H_j , \ i = \{1, 2, 3\}
$$

Где для записи использовались символ Кронекера и сокращение $\partial_i H_j  \equiv \frac  {\partial H_j }{\partial \theta_{i}} $. Если расписать этот градиент по координатам, то получим:

$$
\nabla_{\theta} \mathcal{L}(\theta) =
\begin{pmatrix}
0.2 \cdot \theta_1 + \frac{1}{5}\sum\limits_{j=1}^5 \frac  {\partial H_j }{\partial \theta_{1}} \\
0.2 \cdot \theta_2 + \frac{1}{5}\sum\limits_{j=1}^5 \frac  {\partial H_j }{\partial \theta_{2}} \\
\frac{1}{5}\sum\limits_{j=1}^5 \frac  {\partial H_j }{\partial \theta_{3}}
\end{pmatrix} =
\begin{pmatrix} 0.2 \\0.2 \\0\end{pmatrix} \otimes \theta + \frac15 \sum\limits_{j=1}^5\nabla_{\theta} H_j = \begin{pmatrix} 0.2 \\0.2 \\0\end{pmatrix} \otimes \theta - \frac15 \sum\limits_{j=1}^5 y_j \cdot [M_j < 1] \cdot x_{new,j} =
$$

$$
= \begin{pmatrix} 0.2 \\0.2 \\0\end{pmatrix} \otimes \theta - \frac15 \sum\limits_{j=1}^5 y_j \cdot [y_j \cdot \langle x_{new,j}, \theta \rangle < 1] \cdot x_{new,j}
$$

Тут индекс $j$ опять же обозначает номер семпла; $\otimes$ - поэлементное умножение векторов (операция Адамара, аналог операции * в numpy).

In [1]:
import numpy as np
import yaml

In [2]:
X = np.array([
    [0, 1],
    [1, 1],
    [1, 0],
    [-0.5, 0.5],
    [0, -0.5]
])

y = np.array([1, 1, 1, -1, -1])

theta0 = np.array([1.0, 1.0, 0.0])

lr = 0.1

def f(X, theta):
    theta = np.asarray(theta)
    return (X * theta[:2]).sum(axis=-1) + theta[2]

def loss(X, y, theta):
    theta = np.asarray(theta)
    norm = (theta[:2] ** 2).sum()
    deltas = y * f(X, theta)
    return 0.1 * norm + np.mean(np.maximum(0, 1 - deltas))

print("Prediction:", f(X, theta0))
print("Loss:", loss(X, y, theta0))

Prediction: [ 1.   2.   1.   0.  -0.5]
Loss: 0.5


In [3]:
# Ваш код оптимизации.
def grad_descent(X, y, theta0, lr=0.1):
    X_new = X.copy()
    X_new = np.concatenate((X_new, np.ones((X.shape[0], 1))), axis=1)
    a = np.array([0.2, 0.2, 0])
    theta = np.array(theta0)

    for j in range(1000):
        I = X_new * (((y * (X_new @ theta)) < 1).reshape(-1, 1))
        grad =  -np.sum(I * (y / 5).reshape(-1, 1), axis=0)
        grad_full =  a * theta + grad
        theta -= lr * grad_full

    return theta


theta = grad_descent(X, y, theta0, 0.1)
theta

array([ 1.49999998,  1.        , -0.5       ])

In [4]:
print("Prediction:", f(X, theta))
print("Loss:", loss(X, y, theta))

with open("submission.yaml", "w") as fp:
    yaml.safe_dump({"tasks": [{"task1": {"answer": theta.tolist()}}]}, fp)

Prediction: [ 0.5         1.99999998  0.99999998 -0.74999999 -1.        ]
Loss: 0.4750000000000001
